In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from scipy import stats
from scipy.stats import norm, skew #for some statistics
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, roc_auc_score 
import seaborn as sns  #advanced visualization library
import requests, zipfile, io
import warnings
warnings.filterwarnings('ignore')

In [ ]:
train = pd.read_csv('/content/drive/My Drive/DATACEPT COMP6/train.csv')
test = pd.read_csv('/content/drive/My Drive/DATACEPT COMP6/test.csv')
sample_submission = pd.read_csv('/content/drive/My Drive/DATACEPT COMP6/Sample_sub.csv')

In [ ]:
missing_percentage=(train.isna().sum()/len(train)).sort_values(ascending=False)
missing_percentage

damage_grade                              0.0
has_secondary_use_other                   0.0
has_superstructure_stone_flag             0.0
has_superstructure_mud_mortar_stone       0.0
has_superstructure_adobe_mud              0.0
plan_configuration                        0.0
position                                  0.0
other_floor_type                          0.0
ground_floor_type                         0.0
roof_type                                 0.0
foundation_type                           0.0
land_surface_condition                    0.0
height_percentage                         0.0
area_percentage                           0.0
age                                       0.0
count_floors_pre_eq                       0.0
geo_level_3_id                            0.0
geo_level_2_id                            0.0
geo_level_1_id                            0.0
has_superstructure_cement_mortar_stone    0.0
has_superstructure_mud_mortar_brick       0.0
has_superstructure_cement_mortar_b

In [ ]:
Y=train.damage_grade
ids=test.building_id

In [ ]:
train.head()

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,ground_floor_type,other_floor_type,position,plan_configuration,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,legal_ownership_status,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade
0,914510,21,363,4982,3,30,6,7,t,r,q,f,q,s,d,0,1,0,0,0,0,1,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0,3
1,199801,15,535,9295,2,55,6,4,t,r,n,f,q,s,d,0,1,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0,2
2,538523,17,1119,12390,3,5,6,7,t,r,q,f,q,t,d,0,1,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0,3
3,48420,7,1040,363,3,30,10,7,t,r,n,f,q,t,s,0,1,0,0,0,1,0,0,0,0,0,v,1,1,0,1,0,0,0,0,0,0,0,0,2
4,709794,27,181,4652,4,65,31,12,t,i,x,v,s,t,d,0,0,0,0,1,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0,3


In [ ]:
train=train.drop(["building_id","damage_grade"],axis=1)

In [ ]:
train=pd.get_dummies(train)

# Modelling

In [ ]:
train_X, val_X, train_y, val_y = train_test_split(train,Y , test_size=0.20, random_state=4)

In [ ]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(sparse=False)
enc.fit(train_y.to_numpy().reshape(-1, 1))
encoded_Y = enc.transform(train_y.to_numpy().reshape(-1, 1))
encoded_Y

array([[0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       ...,
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.]])

In [ ]:
enc.fit(val_y.to_numpy().reshape(-1, 1))
en_val_Y = enc.transform(val_y.to_numpy().reshape(-1, 1))


In [ ]:
from keras.utils import to_categorical
encoded_Y = to_categorical(train_y)
en_val_Y = to_categorical(val_y)


In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, LSTM, Activation, MaxPool2D, BatchNormalization

In [ ]:
train.shape

(221510, 68)

In [ ]:
model0=Sequential()
model0.add(Dense(68, activation = "relu"))
model0.add(Dense(512, activation = "relu"))
model0.add(Dropout(0.2))
model0.add(Dense(256, activation = "relu"))
model0.add(Dense(128, activation = "relu"))
model0.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001))
model0.add(Dropout(0.2))
model0.add(Dense(64, activation = "relu"))
model0.add(Dense(4, activation="softmax"))

In [ ]:
from keras.optimizers import SGD
opt = SGD(lr=0.01)
model0.compile(loss = "categorical_crossentropy", optimizer = opt, metrics=['accuracy'])

In [ ]:
train_X2=train_X.drop(["geo_level_1_id","geo_level_2_id","geo_level_3_id"],axis=1)
val_X2=val_X.drop(["geo_level_1_id","geo_level_2_id","geo_level_3_id"],axis=1)

In [ ]:
X = train_X.to_numpy()
Y = train_y.to_numpy()
X_val=val_X.to_numpy()
Y_val=val_y.to_numpy()

In [ ]:
model0.fit(X, encoded_Y,validation_data=(X_val ,en_val_Y), batch_size=64, epochs=5), val_X, train_y, val_y

Train on 177208 samples, validate on 44302 samples
Epoch 1/5
177208/177208 [==============================] - 17s 95us/step - loss: 0.9542 - acc: 0.5563 - val_loss: 0.9214 - val_acc: 0.5678
Epoch 2/5
177208/177208 [==============================] - 15s 87us/step - loss: 0.9210 - acc: 0.5678 - val_loss: 0.9188 - val_acc: 0.5678
Epoch 3/5
177208/177208 [==============================] - 16s 88us/step - loss: 0.9175 - acc: 0.5685 - val_loss: 0.9183 - val_acc: 0.5678
Epoch 4/5
177208/177208 [==============================] - 15s 87us/step - loss: 0.9164 - acc: 0.5687 - val_loss: 0.9163 - val_acc: 0.5678
Epoch 5/5
177208/177208 [==============================] - 15s 87us/step - loss: 0.9155 - acc: 0.5686 - val_loss: 0.9156 - val_acc: 0.5678


(<keras.callbacks.History at 0x7f25ccbfb9e8>,
         geo_level_1_id  ...  legal_ownership_status_w
 202209              25  ...                         0
 27133                6  ...                         0
 67065               21  ...                         0
 199299              21  ...                         0
 182851              13  ...                         0
 ...                ...  ...                       ...
 188967              17  ...                         0
 89588                6  ...                         0
 121329              21  ...                         0
 41925               22  ...                         0
 28993                7  ...                         0
 
 [44302 rows x 68 columns],
 56434     3
 42399     3
 215625    2
 138048    3
 217530    3
          ..
 129384    2
 120705    2
 115397    2
 204974    3
 214138    2
 Name: damage_grade, Length: 177208, dtype: int64,
 202209    2
 27133     2
 67065     2
 199299    3
 182851    2
     

In [ ]:
model0.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_60 (Dense)             (None, 68)                4692      
_________________________________________________________________
dense_61 (Dense)             (None, 512)               35328     
_________________________________________________________________
dropout_9 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_62 (Dense)             (None, 256)               131328    
_________________________________________________________________
dense_63 (Dense)             (None, 128)               32896     
_________________________________________________________________
batch_normalization_2 (Batch (None, 128)               512       
_________________________________________________________________
dropout_10 (Dropout)         (None, 128)             

In [ ]:
import lightgbm as lgb
lgb_model = lgb.LGBMRegressor(colsample_bytree=0.4,
                 learning_rate=0.1,
                 max_depth=12,
                 min_child_weight=1.5,
                 n_estimators=500,                                                                   
                 subsample=0.6,
                 seed=42)
lgb_model.fit(train_X,train_y)
val_pred_lgb = lgb_model.predict(val_X)
print(val_pred_lgb)

[2.07381454 2.34306756 2.56690192 ... 2.2806177  1.90117638 2.44666394]


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

gkf = KFold(n_splits=5, shuffle=True, random_state=42).split(X=train_X, y=train_y)
param_grid = {
    'reg_alpha': [0.1, 0.5],
    'lambda_l1': [0, 1, 1.5],
    'lambda_l2': [0, 1]
    }
gsearch = GridSearchCV(estimator=lgb_model, param_grid=param_grid, cv=2)
lgb_model = gsearch.fit(X=train_X, y=train_y)



In [ ]:
import xgboost as xgb
xgb_model =xgb.XGBClassifier(gamma=0, 
                             learning_rate=0.1, max_depth=8, 
                             n_estimators=500,
                             tree_method='gpu_hist' )
xgb_model=xgb_model.fit(train_X, train_y)
val_pred_xgb = xgb_model.predict_proba(val_X)
#val_mae_xgb = roc_auc_score(val_pred_xgb, val_y)
#print(val_mae_xgb)


In [ ]:
pip install catboost

     |████████████████████████████████| 64.4MB 45kB/s 


In [ ]:
from catboost import CatBoostClassifier
model = CatBoostClassifier(n_estimators=500,
                          learning_rate=0.1,
                          depth=8)
model.fit(train_X,train_y)
val_pred = model.predict(val_X)


0:	learn: 1.0429843	total: 181ms	remaining: 17.9s
1:	learn: 0.9969397	total: 349ms	remaining: 17.1s
2:	learn: 0.9590366	total: 516ms	remaining: 16.7s
3:	learn: 0.9280155	total: 703ms	remaining: 16.9s
4:	learn: 0.9015424	total: 878ms	remaining: 16.7s
5:	learn: 0.8787022	total: 1.04s	remaining: 16.3s
6:	learn: 0.8598665	total: 1.24s	remaining: 16.4s
7:	learn: 0.8428934	total: 1.41s	remaining: 16.3s
8:	learn: 0.8273288	total: 1.58s	remaining: 16s
9:	learn: 0.8141245	total: 1.76s	remaining: 15.8s
10:	learn: 0.8029885	total: 1.93s	remaining: 15.6s
11:	learn: 0.7941250	total: 2.12s	remaining: 15.6s
12:	learn: 0.7852235	total: 2.3s	remaining: 15.4s
13:	learn: 0.7770068	total: 2.47s	remaining: 15.2s
14:	learn: 0.7698292	total: 2.65s	remaining: 15s
15:	learn: 0.7637222	total: 2.82s	remaining: 14.8s
16:	learn: 0.7589206	total: 3.01s	remaining: 14.7s
17:	learn: 0.7529283	total: 3.19s	remaining: 14.5s
18:	learn: 0.7486150	total: 3.37s	remaining: 14.4s
19:	learn: 0.7450012	total: 3.54s	remaining: 1

In [ ]:
val_pred

array([[2],
       [2],
       [3],
       ...,
       [2],
       [2],
       [2]])

# Testing

In [ ]:
test=test.drop("building_id",axis=1)

In [ ]:
test=pd.get_dummies(test)

In [ ]:
train.shape

(221510, 68)

In [ ]:
submission_result0 = model0.predict(test.to_numpy())

In [ ]:
submission_result0

array([[2.88574025e-03, 9.20147598e-02, 5.58402419e-01, 3.46697122e-01],
       [1.13134643e-04, 1.18781865e-01, 5.35692573e-01, 3.45412433e-01],
       [2.46435343e-06, 1.09315626e-01, 5.67672908e-01, 3.23008984e-01],
       ...,
       [4.79874114e-04, 1.08868957e-01, 5.41071236e-01, 3.49579901e-01],
       [2.98381610e-06, 1.09161995e-01, 5.66659093e-01, 3.24175924e-01],
       [8.07496253e-05, 1.05705015e-01, 5.51601768e-01, 3.42612475e-01]],
      dtype=float32)

In [ ]:
l=[]
for i in range(len(submission_result0)):
  if submission_result0[i]==[1., 0., 0., 0.]:
    l.append(0)
  if submission_result0[i]==[0., 1., 0., 0.]:
    l.append(1)
  if submission_result0[i]==[0., 0., 1., 0.]:
    l.append(2)
  if submission_result0[i]==[1., 0., 0., 1.]:
    l.append(3)


In [ ]:
# make predictions which we will submit.
l=[] 
test_preds_lgb = lgb_model.predict(test)

for i in range(len(test_preds_lgb)):
  k=test_preds_lgb[i]
  if k <0.5:
    l.append(0)
  if k>0.5 and k<1.5:
    l.append(1)
  if k >1.5 and k<2.5:
    l.append(2)
  if k >2.5:
    l.append(3)
test_preds_lgb=l

# The lines below shows how to save predictions in format used for competition scoring
# Just uncomment them.
output = pd.DataFrame({'building_id': ids, 'damage_grade': (test_preds_lgb)})
output.to_csv('/content/drive/My Drive/DATACEPT COMP6/sub_lgb.csv', index=False)

In [ ]:
# make predictions which we will submit.
l=[] 
test_preds_xgb = xgb_model.predict(test)

for i in range(len(test_preds_xgb)):
  k=test_preds_xgb[i]
  if k <0.5:
    l.append(0)
  if k>0.5 and k<1.5:
    l.append(1)
  if k >1.5 and k<2.5:
    l.append(2)
  if k >2.5:
    l.append(3)
test_preds_xgb=l

# The lines below shows how to save predictions in format used for competition scoring
# Just uncomment them.
output = pd.DataFrame({'building_id': ids, 'damage_grade': (test_preds_xgb)})
output.to_csv('/content/drive/My Drive/DATACEPT COMP6/sub_xgb.csv', index=False)

In [ ]:
# make predictions which we will submit.
l=[] 
test_preds_cat = model.predict(test)

for i in range(len(test_preds_cat)):
  k=test_preds_cat[i]
  if k <0.5:
    l.append(0)
  if k>0.5 and k<1.5:
    l.append(1)
  if k >1.5 and k<2.5:
    l.append(2)
  if k >2.5:
    l.append(3)
test_preds_cat=l

# The lines below shows how to save predictions in format used for competition scoring
# Just uncomment them.
output = pd.DataFrame({'building_id': ids, 'damage_grade': (test_preds_cat)})
output.to_csv('/content/drive/My Drive/DATACEPT COMP6/sub_cat.csv', index=False)

In [ ]:
# make predictions which we will submit.
l=[] 
test_preds = 0.5*xgb_model.predict(test)+0.5*lgb_model.predict(test)

for i in range(len(test_preds)):
  k=test_preds[i]
  if k <0.5:
    l.append(0)
  if k>0.5 and k<1.5:
    l.append(1)
  if k >1.5 and k<2.5:
    l.append(2)
  if k >2.5:
    l.append(3)
test_preds=l

# The lines below shows how to save predictions in format used for competition scoring
# Just uncomment them.
output = pd.DataFrame({'building_id': ids, 'damage_grade': (test_preds)})
output.to_csv('/content/drive/My Drive/DATACEPT COMP6/m5alet.csv', index=False)